# Numpy

[Numpy](https://www.numpy.org/) is the foundation of most of what you will do with scientific python. Modules like [scipy](https://www.scipy.org/), [pandas](https://pandas.pydata.org/) and [scikit-learn](https://scikit-learn.org/stable/) are built on top of numpy. It is the linga franca for most numerical work in python.

In [1]:
import numpy as np

When you are first introduced to python, one of the big selling points is that it isn't statically typed. You can do things like

In [2]:
a = 1
a = 'apple'
print(a)

apple


And you won't get complaints from python about a being an integer. This is a _big_ advantage for python, and it works with collections as well

In [3]:
myList = [1, 3., 'Ian', [range(3)], {'not': 'a good idea'}]
myList

[1, 3.0, 'Ian', [range(0, 3)], {'not': 'a good idea'}]

Lists in python are about as general as they could be. This is very flexible and it lets you do some fancy things, but it has a price. The Python interpreter can't make any assumptions about what will come next in a list; everything has to be treated as a generic object. Python does a good job of hiding the complexity of doing this, but as the lists get longer and more complex the overhead gets larger, and eventually perfomance becomes unacceptable.

One solution to this is to use a statically typed language like C[<sup>1</sup>](#fn1 "footnote and tooltip 1"). There the burden of figuring out object types is left to the programmer, and the compiler can be much more efficient operating on them. A good example would be an array of `double`s. In memory, these will be allocated contiguously so when you need to jump to the 1402th double, you can do it with very simple arithmetic. Python has a much harder time because the memory allocated for your list could be a horrible mixture of all the different things you've stuffed in there. 

## The `ndarray`

Numpy attempts let you keep the advantages of Python without sacrificing the speed of static typing by adding the concept of homogeneous collections to python: `ndarray`s. The `ndarray` is the foundational concept in numpy, it is an array object which represents a multidimensional, homogeneous array of fixed-size items and most commonly these items will be numbers.

In [4]:
%%timeit
for i in range(1000000):
    i*i

75.7 ms ± 3.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%timeit np.arange(1000000)**2

4.85 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


`ndarray`s look like python lists, but they are fundamentally different, e.g.

In [7]:
a = [1, 2, 3, 4]
b = [5, 6, 7, 8]
a+b

[1, 2, 3, 4, 5, 6, 7, 8]

In [8]:
na = np.array([1, 2, 3, 4])
nb = np.array([5, 6, 7, 8])
na + nb

array([ 6,  8, 10, 12])

`numpy` was able to assume that the things in the `ndarray` were the compatible types and vectorize the addition, if we want the same thing with python lists, we have to jump through some hoops

In [9]:
[ i + j for i, j in zip(a, b) ] 

[6, 8, 10, 12]

_If you dig deep enough in some of the numpy/scipy code you will find that the actual instructions you are executing were compiled from fortran and C. In general you can pass things quite easily between existing libraries and python, but fortran and C use different storage orders for multi-dimensional arrays so you have to be a little bit careful_

Let's take a look at what actually makes up an `ndarray`.

In [10]:
na = np.array([1,2,3,4,5])
na

array([1, 2, 3, 4, 5])

In [13]:
attr_names = [attr_name for attr_name in dir(np.ndarray)
              if not callable(getattr(np.ndarray, attr_name)) 
              and not attr_name[:2] == '__']

attr_help = {a : getattr(np.ndarray, a).__doc__.split('\n')[0] for a in attr_names}

In [15]:
attr_help

{'T': 'The transposed array.',
 'base': 'Base object if memory is from some other object.',
 'ctypes': 'An object to simplify the interaction of the array with the ctypes',
 'data': "Python buffer object pointing to the start of the array's data.",
 'dtype': "Data-type of the array's elements.",
 'flags': 'Information about the memory layout of the array.',
 'flat': 'A 1-D iterator over the array.',
 'imag': 'The imaginary part of the array.',
 'itemsize': 'Length of one array element in bytes.',
 'nbytes': 'Total bytes consumed by the elements of the array.',
 'ndim': 'Number of array dimensions.',
 'real': 'The real part of the array.',
 'shape': 'Tuple of array dimensions.',
 'size': 'Number of elements in the array.',
 'strides': 'Tuple of bytes to step in each dimension when traversing an array.'}

By knowing these attributes, numpy can use some of the same tricks that statically typed languges use because the Python interpreter can now infer the memory layout.

In [16]:
a = np.zeros(10, dtype=float)
a.nbytes

80

numpy has some convenience methods for creating new ndarrays. As you saw above, one way to create an ndarray is to pass a list with the values to `np.array`. Here are some others...

Using `np.array` directly

In [18]:
a1 = np.array([0, 1, 1, 2, 3, 5, 8, 13])
a1

array([ 0,  1,  1,  2,  3,  5,  8, 13])

`np.arange` will generate numbers between limits

In [19]:
a2 = np.arange(0, 100)
a2

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [20]:
a22 = np.arange(0, 10, dtype=float)
a22

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

`np.linspace` is extremely useful, you tell it where to start and stop and how many samples you need and linspace does the rest. Here we will create numbers 100 numbers between 0 (inclusive) and 1, linearly spaced

In [21]:
li1 = np.linspace(0, 1, 20)
li1

array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])

_Exercise: Use linspace to generate 25 of the values between 1 and 5, excluding the endpoint_

In [24]:
np.linspace(0, 1, 20, endpoint=True)

array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])

The `np.zeros` function will generate an `ndarray` full of zeros. The first argument is the shape which you can give as an integer (for 1d arrays) or a sequence (for n-dimensional arrays). You can also pass the `dtype=` argument to tell it what type of number you are looking for.

In [26]:
# z1 100 integer zeros
z1 = np.zeros(100, dtype=int)

# z2 a 5,5 array of float64 zeros
z2 = np.zeros((5, 5), dtype=float)
z2

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

The `np.ones` function does something similar, but with unit value

In [27]:
# o1 100 integers ones
o1 = np.ones(100, dtype='float64')
o1

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [28]:
# o2 a 5x5 ndarray of complex128 one values. First argument is shape sequence
o2 = np.ones((5, 5), dtype='complex128')
o2

array([[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j]])

The `np.eye` function generates a 2D array with ones down the diagonal, zeros elsewhere

In [29]:
# e1 a 5x5 array with ones down the diagonal
e1 = np.eye(5, dtype=np.float64)
e1

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

_**Exercise**: The diag function lets you specify diagonal arrays by giving the non-zero entries down the diagonal. Try creating a 5x5 array with ones down the diagonal. Have a look at the help and see if you can do the same thing with the ones offset above and below the diagonal_

In [33]:
np.diag([1,1,1,1], k=1) + np.diag([1,1,1,1], k=-1)

array([[0, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0]])

We'll come back to the `numpy.random` module later on, but it has some useful convenience methods. `np.random.randint` returns random integers. Take a look at the help on the method then create a name `r1` with an array of `4x5` random numbers between `0` and `10`.


In [34]:
# r1 a 4x5 array of random integers between 0 & 10, 
np.random.randint(0, 10, size=(4, 5))

array([[2, 3, 1, 5, 0],
       [1, 1, 1, 5, 9],
       [2, 8, 1, 2, 2],
       [8, 7, 7, 1, 0]])

Numpy floats have representations for certain special values `np.nan`, `np.Inf` etc. These objects have well defined comparisons

In [35]:
np.NaN == np.NaN

False

In [36]:
np.NINF < 0

True

In [37]:
np.NAN < np.Infinity

False

When you have an `ndarray` of one type, you can use the `astype` method to return a copy casted to another type

In [38]:
e1.astype(int)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])

## Indexing and Slicing

Now that we have some `ndarray`s to play with, lets look at using them. Of course, `ndarray`s are zero indexed


In [39]:
# First element of a2
a2[0]

0

The usual negative number notation works for counting from the end

In [40]:
# 2nd to last element of a2
a2[-2]

98

We can update `ndarray` in place by index


In [42]:
a2[-1] = 0
a2[-1]

0

Slicing returns subarrays of the original `ndarray`. Crucially it does this inexpensively by returning a "view" on the data rather than copying it. This is much more efficient and relies on numpy is using it's knowledge of the memory layout to return only the things you ask for. This is a general tactic used by numpy, if it _can_ return a view rather than a copy it _will_! If you really need a copy, try the `.copy()` method on `ndarrays`.

Slicing uses the same notation as core python: `[start:stop:step]` where `start` is inclusive and `stop` is exclusive.

In [43]:
# Values between 3 and 19 of a2
a2[3:19]

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18])

In [44]:
# Every third value between 3 and 19
a2[3:19:3]

array([ 3,  6,  9, 12, 15, 18])

Using negative is allowed for all three parts of the slice, but for the step you have to think a bit

In [45]:
# Values between -10 and -2
a2[-10:-2]

array([90, 91, 92, 93, 94, 95, 96, 97])

Notice that the first argument of the slice is still inclusive and the second is not. If we omit a value when specifying the slice `start` defaults to 0, `end` defaults to the last element and `step` defaults to 1.

In [46]:
a2[::3]

array([ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48,
       51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96,  0])

For multi-dimensional arrays the indexing notation is similar

In [47]:
b = np.arange(100)
b.shape = (10, 10)
b

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

How about row 0, column 3 (remember python is 0 indexed)

In [48]:
b[0,3]

3

In [49]:
b[:, -1]

array([ 9, 19, 29, 39, 49, 59, 69, 79, 89, 99])

Or the fifth column of the first two rows

In [50]:
b[0:2, 4]

array([ 4, 14])

In [51]:
b[0:2, 4:6]

array([[ 4,  5],
       [14, 15]])

In [52]:
b[::2, 9]

array([ 9, 29, 49, 69, 89])

_**Exercise**: Create a 2d array with 1 on the border and 0 inside_

In [54]:
z = np.zeros((5,5))
z[:1,:] = z[:, :1] = 1
z[-1:,:] = z[:, -1:] = 1
z

array([[1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 1., 1., 1.]])

In [55]:
z1 = np.zeros((3,3))
np.pad(z1, pad_width=((1,1), (1,1)), constant_values=1)

array([[1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 1., 1., 1.]])

### Fancy Indexing

Fancy Indexing is the idea of using another array of indices, it is useful when the combinations you want to select become a bit more complicated. We'll see more of this in pandas but as a few quick examples...

In [62]:
v1=np.arange(27)[::-1]
v1

array([26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,
        9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [63]:
v1[[1, 4, 6]]

array([25, 22, 20])

In [64]:
v2 = v1.reshape(3,9)
v2

array([[26, 25, 24, 23, 22, 21, 20, 19, 18],
       [17, 16, 15, 14, 13, 12, 11, 10,  9],
       [ 8,  7,  6,  5,  4,  3,  2,  1,  0]])

In higher dimensions think of zipping together the arguements, e.g. first row (index 0), fourth column (index 3) column

In [67]:
v2[[0, 1, 1], [3, 7, 8]]

array([23, 10,  9])

You can also index with logical values

In [69]:
rng = np.random.default_rng(42)
ar = rng.integers(0, 10, size=25)
ar

array([0, 7, 6, 4, 4, 8, 0, 6, 2, 0, 5, 9, 7, 7, 7, 7, 5, 1, 8, 4, 5, 3,
       1, 9, 7])

In [74]:
# Negate any values between 3 and 8
# Just be careful to use parentheses to avoid bitwise operations
ar[(3 < ar) & (ar < 8)] *= -1
ar

array([ 0, -7, -6, -4, -4,  8,  0, -6,  2,  0, -5,  9, -7, -7, -7, -7, -5,
        1,  8, -4, -5,  3,  1,  9, -7])

The `np.argmax` and `np.argmin` functions can return the maximum or minimum value along some axis and can be used together with fancy indexing to grab values of interest

In [80]:
ar.shape = (5, 5)
np.argmax(ar)

11

In [82]:
np.unravel_index(11, ar.shape)

(2, 1)

`argmax` returns the index for a flattened copy of the array, but the `unravel_index` function can help us translate that to the position we actually want (we're deliberately ignoring functions like `max` here).

In [83]:
ar[np.unravel_index(np.argmax(ar), ar.shape)]

9

*N.B. Fancy indexing usually creates copies of the `ndarray` because you usually can't reconstruct the selection with simple algebra*

## Reshaping Arrays

Sometimes it is convenient to reshape arrays. I did this above by setting the `.shape` attribute but numpy arrays also have a reshape method.

In [84]:
c = np.arange(27)
c

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26])

`reshape` expects a sequence as the first argument (e.g. a tuple) so

In [85]:
d = c.reshape((3, 9))
d

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23, 24, 25, 26]])

One argument to reshape can be `-1`, in which case the number is determined by the number and size of remaining dimensions.

In [87]:
# A 256x256 array of pixels with values between 0 and 4 reshaped to 2 dimensional
w, h = 256, 256
I = np.random.randint(0, 4, (h, w, 3)).astype(np.ubyte)
I.reshape(-1, 3).shape

(65536, 3)

Reshaping isn't enough to provoke numpy to copy the data, all it needs to do is make a new view on the same data

In [89]:
d.base is c

True

Another common reshaping task is to add dimension(s) to an existing array. numpy has a special `newaxis` object for this task. This is a powerful idea when combined with numpy's broadcasting rules (see below).

In [90]:
e = np.arange(10)
# 10 x 1
e[:, np.newaxis]

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [91]:
# 1 x 10
e[np.newaxis,:]

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [92]:
# 1 x 1 x 10
e[np.newaxis, np.newaxis, :]

array([[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]])

Sometimes it is necessary to go back and forth between flat (1d) and multi-dimensional views of an array. The `np.ravel`, `np.flat` functions and `ndarray.flatten` method as well as things like `np.unravel_index` can help with that.

## Stacking & Splitting ndarrays

You can combine general ndarrays with the `np.concatenate` and split them with `np.split`. There are also a number of convenience methods for commonly used shapes.

 * `_r`
 * `_c`
 * `hstack`
 * `vstack`
 * `hsplit`
 * `vsplit`

`_r` translates slice objects to concatenation along the first axis.

In [94]:
np.r_[np.array([1,2,3]), 0, 0, np.array([4,5,6])]


array([1, 2, 3, 0, 0, 4, 5, 6])

`_c` translates slice objects to concatenation along the second axis.

In [95]:
np.c_[np.array([1,2,3]), np.array([4,5,6])]

array([[1, 4],
       [2, 5],
       [3, 6]])

`hstack`  and `vstack` do something similar, all 4 are basically ways of calling the more generic `np.concatenate` or `np.stack` functions for specific cases

In higher dimensions `concatenate` or `stack` should do what you need, but you need to manually tell it which axis to use for the stacking.

In [96]:
np.concatenate((np.array([1,2,3]), [0], [0], np.array([4, 5, 6])))

array([1, 2, 3, 0, 0, 4, 5, 6])

In [97]:
np.concatenate((np.array([[1, 2, 3]]).T, np.array([[4, 5, 6]]).T), axis=1)

array([[1, 4],
       [2, 5],
       [3, 6]])

`np.split` goes in the opposite direction. It will try to produce sub-arrays of equal size. Again there are `hsplit` and `vsplit` variants for common use cases.

In [98]:
np.split(np.arange(64).reshape((8, 8)), 2, axis=0)

[array([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31]]),
 array([[32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [56, 57, 58, 59, 60, 61, 62, 63]])]

In [99]:
np.split(np.arange(64).reshape((8, 8)), 2, axis=1)

[array([[ 0,  1,  2,  3],
        [ 8,  9, 10, 11],
        [16, 17, 18, 19],
        [24, 25, 26, 27],
        [32, 33, 34, 35],
        [40, 41, 42, 43],
        [48, 49, 50, 51],
        [56, 57, 58, 59]]),
 array([[ 4,  5,  6,  7],
        [12, 13, 14, 15],
        [20, 21, 22, 23],
        [28, 29, 30, 31],
        [36, 37, 38, 39],
        [44, 45, 46, 47],
        [52, 53, 54, 55],
        [60, 61, 62, 63]])]

See also `np.tile`

In [100]:
np.tile(np.array([[1,0],[0,1]]), (4, 4))

array([[1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 1]])

## Universal Functions (ufuncs)

The real reason for using numpy is so you can do numerical operations, _quickly_. Python uses a concept called `ufuncs` or universal function. A ufunc is a function which operates on `ndarrays` element-by-element. More formally, a `ufunc` is a vectorized wrapper around a function which can do a transformation on an `ndarray` and produces another `ndarray`. This element by element behaviour is fundamentally different from the usual python behaviour.

The key to writing fast numeric python code is: **Avoid for & while loops as far as you can, use numpy ufuncs as far as possible**


Lets start with basic arithmetic operations. Numpy can use it's internal broadcasting to do these quickly and efficiently

The usual operations are available

  * +: addition
  * -: subtraction
  * *: multiplication
  * /: division
  * //: integer division
  * **: power operator
  * %: modulo

Remember operations are element by element, and you can build up more complicated expressions as you go

In [101]:
la = np.linspace(0, 1, 100)

(la ** 2 + la) / (la + 1)

array([0.        , 0.01010101, 0.02020202, 0.03030303, 0.04040404,
       0.05050505, 0.06060606, 0.07070707, 0.08080808, 0.09090909,
       0.1010101 , 0.11111111, 0.12121212, 0.13131313, 0.14141414,
       0.15151515, 0.16161616, 0.17171717, 0.18181818, 0.19191919,
       0.2020202 , 0.21212121, 0.22222222, 0.23232323, 0.24242424,
       0.25252525, 0.26262626, 0.27272727, 0.28282828, 0.29292929,
       0.3030303 , 0.31313131, 0.32323232, 0.33333333, 0.34343434,
       0.35353535, 0.36363636, 0.37373737, 0.38383838, 0.39393939,
       0.4040404 , 0.41414141, 0.42424242, 0.43434343, 0.44444444,
       0.45454545, 0.46464646, 0.47474747, 0.48484848, 0.49494949,
       0.50505051, 0.51515152, 0.52525253, 0.53535354, 0.54545455,
       0.55555556, 0.56565657, 0.57575758, 0.58585859, 0.5959596 ,
       0.60606061, 0.61616162, 0.62626263, 0.63636364, 0.64646465,
       0.65656566, 0.66666667, 0.67676768, 0.68686869, 0.6969697 ,
       0.70707071, 0.71717172, 0.72727273, 0.73737374, 0.74747

**Example**: Try to calculate the terms of this sum as an `ndarray`
$$
\sqrt{12}\sum_{k=0}^{10}\frac{(-3)^{-k}}{2k+1}
$$

_Hints_: Think term by term. `np.arange(10)` will give you an `ndarray` of k values, next raise `-3. * np.ones(k.shape)` to those powers. If you can calculate the terms you can use the `.sum()` method to sum them all up. How close to $\pi$ did you get?

In [108]:
k = np.arange(11)
((-3. * np.ones(k.shape))**-k / (2 * k + 1)).sum() * np.sqrt(12)

3.141593304503081

The operators we were using `+,-,/,...` actually correspond to functions (`ufuncs`)

|operator|function|description|
|--------|--------|-----------|
| + | np.add | Addition |
| - | np.subtract | Subtraction |
| - | np.negative | Unary negation |
| * | np.multiply | Multiplication |
| / | np.division | Ordinary floating point division |
| // | np.floor_divide | floor (integer) division |
| % | np.mod | Modulo/Remainder division |

You can use either syntax, but in the function notation there are lots more functions to play with e.g.

| function | description |
|----------|-------------|
| np.sin   | sin function |
| np.cos   | cos function |
| np.tan   | tan function |
| np.abs   | absolute value |
| np.exp   | exponential |
| np.log   | natural log |
| np.log2  | log base 2 |
| np.log10 | log base 10 |
|  ...     |    ...      |


In [110]:
p1 = np.linspace(0, 2*np.pi, 25)
p2 = np.sin(p1)
p2

# p2 sin of p1

array([ 0.00000000e+00,  2.58819045e-01,  5.00000000e-01,  7.07106781e-01,
        8.66025404e-01,  9.65925826e-01,  1.00000000e+00,  9.65925826e-01,
        8.66025404e-01,  7.07106781e-01,  5.00000000e-01,  2.58819045e-01,
        1.22464680e-16, -2.58819045e-01, -5.00000000e-01, -7.07106781e-01,
       -8.66025404e-01, -9.65925826e-01, -1.00000000e+00, -9.65925826e-01,
       -8.66025404e-01, -7.07106781e-01, -5.00000000e-01, -2.58819045e-01,
       -2.44929360e-16])

In [111]:
q1 = np.linspace(0, 1, 10) + np.linspace(1, 2, 10)*1j
q1

array([0.        +1.j        , 0.11111111+1.11111111j,
       0.22222222+1.22222222j, 0.33333333+1.33333333j,
       0.44444444+1.44444444j, 0.55555556+1.55555556j,
       0.66666667+1.66666667j, 0.77777778+1.77777778j,
       0.88888889+1.88888889j, 1.        +2.j        ])

In [112]:
np.abs(q1)

array([1.        , 1.11665285, 1.24225999, 1.37436854, 1.5112745 ,
       1.65178542, 1.79505494, 1.94047213, 2.08758825, 2.23606798])

## Aggregate Functions

Aggregate functions take an `ndarray` and reduce it along one (or more) axes. An example would be taking an array of numbers and calculating the mean value...

In [113]:
r1 = np.linspace(0, 10, 100)
r1.mean()

5.0

But there are lots of aggregate functions

  * `min`: Minumum value
  * `max`: Maximum value
  * `sum`: Sum values
  * `prod`: Product of values
  * `mean`: Arthmetic mean
  * `std`: Standard deviation
  * `var`: Variance
  * `argmin`: indices of the minimum value
  * `argmax`: indices of the maximum value
  * `all`: is a condition true in all elements
  * `any`: is a condition true in any elements
  
  The default is to reduce along all axes, if you want to reduce along a specific axis you can pass that as an argument (the axes you specify are the ones which get squashed) 

In [114]:
s1 = np.arange(50)
s2 = s1.reshape(5,10)
s2

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

In [116]:
s2.mean(axis=0)

array([20., 21., 22., 23., 24., 25., 26., 27., 28., 29.])

For binary operations (e.g. addition) you can also do reduction, so starting from

[1, 3, 5, 7, 9]

`np.add.reduce` will add 1 to 3, add that to 5 and so on, ...

In [117]:
t1 = np.arange(1, 10, 2)
np.add.reduce(t1)

25

N.B. In this case we used a function from the module and passed in our ndarray. In this case this is the same as `.sum()`

_**Exercise**: Find the value closest to a particular number in an array (try using `np.abs` and `np.argmin`)_

In [122]:
rng = np.random.default_rng(47)
a = rng.normal(size=50)
scalar=1

a[np.argmin(np.abs(a - scalar))]

1.0090560424996242

Numpy also has lots of other utility functions, here are a few commonly used ones

  * `np.bincount`: Count number of occurences of each value in an array of non negative ints
  * `np.allclose`: Check if two arrays are equal within some tolerance
  * `np.pad`: Padding arrays
  * `np.unique`: Find unique elements within an array
  * `np.percentile`: Compute percentiles along some axis
  * `np.sort`: Sort (various algorithms), `np.argsort` returns indices of for sorting
  * `np.where`: Return elements matching some condition
  
and __many__ others.

## Broadcasting

`numpy` is at it's most efficient when it is operating element by element, but not all arrays are the same size. To work around this, `numpy` implements a set of rules under the name of `broadcasting` to make `ndarray`s conform whenever possible. This is great news; it means you don't have to worry about doing that yourself, but it is important to understand the rules so that you know how `numpy` will behave when combining differnt shaped `nbdarrays`. To get the idea, think of
```python
np.arange(10) * 5
```
`numpy` wants to operate element-by-element, but `5` isn't an `ndarray`, it's just a number. If we could prompte 5 to be a `1d` narray and put 5's in in every place `numpy` would be happy. This is the basic idea of broadcasting, explicitly

1. Given two arrays of different dimensions, prepend 1, to the shape of the smaller array
1. Dimensions of size 1 are repeated (without copying) as often as needed to make shapes conform


In [123]:
a = np.arange(15)
a = a.reshape(3, 5)
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [125]:
b = np.arange(5)
b

array([0, 1, 2, 3, 4])

If I want to multiply these two `ndarray`s, `b` has the smaller dimensions (1 vs. 2) so a dimension of length one will be prepended to `b`. `b` will then be repeated 3 times to conform with the shape of a.

In [126]:
a * b

array([[ 0,  1,  4,  9, 16],
       [ 0,  6, 14, 24, 36],
       [ 0, 11, 24, 39, 56]])

Just to be explicit, that operation does something lie

In [129]:
btmp = np.repeat(b[np.newaxis, :], 3, axis=0)
a *  btmp


array([[ 0,  1,  4,  9, 16],
       [ 0,  6, 14, 24, 36],
       [ 0, 11, 24, 39, 56]])

To take a more complicated example...

In [131]:
a = np.arange(24).reshape(2,1,3,4)
b = np.arange(4).reshape(1,1,4)

The rules above imply that the dimensions are compared from right-to-left. The last dimension in both `a` and `b` is 4 so nothing needs to be done there. The second to last dimension of `a` is `3` and of `b` is 1, so b will be repeated 3 times in that dimension. The third to last dimension is one for both arrays so nothing needs to be done. `b` doesn't have any more dimensions so the second rule says a 1 should be prepended, giving it be shape `(1, 1, 3, 4)`, finally, the second rule says that the first dimension of `b` should be repeated twice giving it shape `(2, 1, 3, 4)`. This gives the two arrays conformable shapes so the can be added element by element

In [132]:
a+b

array([[[[ 0,  2,  4,  6],
         [ 4,  6,  8, 10],
         [ 8, 10, 12, 14]]],


       [[[12, 14, 16, 18],
         [16, 18, 20, 22],
         [20, 22, 24, 26]]]])

In [133]:
a

array([[[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]],


       [[[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]])

## Random

Numpy has a few important submodules but `np.random` is probably the most important. As you might expect, it lets you work with random numbers, but in addition to simple random numer generators, it can sample from different distributions (30+ available), handle permutations and do lots of other handy things.

`numpy.random` uses the concept of a `Generators` to implement sampling. The idea is you create a generator object then call methods on that generator to sample from the various distributions. The original `Generator` will normally get it's entropy from a (hopefully reliable source) then you can keep asking it for the `__next__` random elements distributed however you need.

(The generator interface to `numpy.random` is relatively recent, occasionally you will still see me explicity call functions of the submodule).

In [136]:
rng = np.random.default_rng(47)

Here we've seeded the `Generator` with a specific value so the results are reporoducible but normally you would just call `np.random.default_rng()` to get a random value from the OS. Now we can sample from various distributions

In [139]:
rng.normal(5.0, 1.0, (64, 64))

array([[4.45735342, 4.64335811, 4.30406865, ..., 5.42167114, 6.38526465,
        4.13354413],
       [8.00665695, 5.92079181, 5.47152572, ..., 5.17898583, 5.508444  ,
        5.02875987],
       [5.32103314, 6.83178148, 7.3340648 , ..., 4.73805631, 5.75644033,
        3.65266441],
       ...,
       [5.20292953, 5.77532456, 6.21447441, ..., 6.15132939, 5.68308763,
        4.25661547],
       [5.27246854, 3.99620872, 3.67969802, ..., 5.7187088 , 5.37434723,
        4.07115524],
       [5.81618402, 5.68064717, 5.48094715, ..., 5.52649621, 5.40168783,
        5.70795698]])

The binomial distribution

In [140]:
rng.binomial(10, .5, size=20)

array([7, 6, 6, 3, 5, 6, 6, 5, 4, 4, 6, 3, 4, 5, 4, 7, 4, 6, 6, 5])

5 random integers between 10 and 20 (discrete uniform)

In [141]:
rng.integers(10, 20, 5)

array([10, 19, 11, 17, 17])

`np.random` also has functions for shuffling arrays in-place (`np.shuffle`) and selecting elements at random from `ndarrays` (`np.choice`)

In [142]:
a = np.arange(10)
np.random.shuffle(a)
a

array([5, 2, 4, 1, 7, 9, 6, 3, 8, 0])

In [144]:
a = np.arange(10)
np.random.choice(a, 3, replace=True)

array([4, 1, 8])

You can also weight the selections in `np.choice` with probabilities. e.g. Here are the letter frequencies of ordinary english text.

In [148]:
letter_freq = {' ': 0.19266420666588144,
 'e': 0.09680968984305797,
 't': 0.07140241019840815,
 'a': 0.06361581392196947,
 'o': 0.06183938572048604,
 'i': 0.05349452953695084,
 'n': 0.0521037201730283,
 'h': 0.051232447485652234,
 's': 0.049280151278754014,
 'r': 0.04524648142979075,
 'd': 0.03375374929612461,
 'l': 0.03124157971419029,
 'u': 0.02392934301198968,
 'm': 0.021518821910249234,
 'w': 0.020208685943305965,
 'c': 0.02004895261728702,
 'f': 0.016262143363080305,
 'y': 0.016250849087503207,
 'g': 0.013559584564274916,
 'p': 0.012933559003715817,
 ',': 0.012259129404969158,
 '.': 0.01200420147051468,
 'b': 0.011160357738111564,
 'v': 0.0076220225466009876,
 'k': 0.006392559976636984,
 'x': 0.001353699601312072,
 'j': 0.0008260955850676769,
 'q': 0.0006663622590487316,
 'z': 0.00031946665203789066
}

We can use those relative frequency values as probabilities (they sum to ~1) and generate a random sample of letters

In [149]:
letters = np.fromiter(letter_freq.keys(),dtype='<U1')
probabilities = np.fromiter(letter_freq.values(), dtype=float)
chosen = np.random.choice(letters, 1000, p=probabilities)
''.join(chosen)

'o r gylos etea eav sen   e rdiritod tndwldraehad du  brostsksvwfw nnw hh  roneeogaow  hw maissnvatdkfufgi e iiaetp aiitai,, nirebo oin,woi hi aeiuwc ughyh i iekmr kibyiladlunearhanst yrha ehohnnavlhserg  be pusuw t  anms oha ntapnus ro.se  o  prwhkt lr  nkaoou knia eeet sotemprmt oa iiuudongaswis iuecedn t h oilne  uih oo sh spptufo  u hgyrphaisnne.  c o rab u edlohieo eahey  eeatearlli l   ai tsse   sit dhj  ah    f e crv detefsuyo neofsei i ctt.gengr thd toagi cegdni sb pe od i eteeoto ne  olsi.kgt,hhttehti hsnatinamtkenh bo absa  ec  wo,ekenkb eoniuonn h enecih g bfthol h  etdnybsnr auwwti  dtgeus,rf  eattiute c dfluy c o nt  doolaitd  h  eefog nnytouddtyd tree tsoo eiluuporeinawiboaoae  olk dhout.eire  o do aad  eeoye e srfhengt iien rti ed xrd  te ur nyhhjautfer aaoshmtoentr s   b,a.h.ara nlueneee,so cx se ar q  ieh sohtr aa ebuwsronutalinikeaa as  gouliba hie nhtmor   otueewo ntrncpe he ighiuteth ybtls   tcr,fahd t  rot  ou  e s.s hhtwe a in eenoseimhxe  sdce ,s,tsoaareoeagoaehp

## See Also

  * [np.linalg](https://numpy.org/doc/stable/reference/routines.linalg.html): Linear algebra operations
  * [np.fft](https://numpy.org/doc/stable/reference/routines.fft.html): Fast Fourier Transforms
  * [Sorting, Searching and Counting](https://numpy.org/doc/stable/reference/routines.sort.html): Sorting